In [22]:
import argparse
import logging
import os
import pprint
import random
from tqdm import tqdm
import sys
# 讓 Python 能找到上層路徑的模組
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import warnings
import numpy as np
import torch
import torch.backends.cudnn as cudnn
import torch.distributed as dist
from torch.utils.data import DataLoader
from torch.optim import AdamW
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

from dataset.LayeredDepth_Syn import LayeredDepth_Syn
from depth_anything_v2.dpt_finetune_1 import DepthAnythingV2
from util.dist_helper import setup_distributed
from util.loss import SiLogLoss
from util.metric import eval_depth
from util.utils import init_log

In [23]:
model = DepthAnythingV2model = DepthAnythingV2(encoder='vitl', features=256, out_channels=[256, 512, 1024, 1024])
ckpt = torch.load("/home/lyc/research/Depth-Anything-V2/metric_depth/result_finetune_1/latest_epoch30.pth", map_location="cuda")
print(type(ckpt))
print(ckpt.keys())
model.load_state_dict(ckpt["model"])
model = model.cuda()    # 或 model.to('cuda')
model.eval()

<class 'dict'>
dict_keys(['model', 'optimizers', 'epoch', 'previous_best'])


DepthAnythingV2(
  (pretrained): DinoVisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
      (norm): Identity()
    )
    (blocks): ModuleList(
      (0-23): 24 x NestedTensorBlock(
        (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (attn): MemEffAttention(
          (qkv): Linear(in_features=1024, out_features=3072, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=1024, out_features=1024, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (drop): Dropout(p=0.0, inplace=Fal

In [ ]:
import matplotlib.pyplot as plt
import cv2

save_dir = "val_vis"
os.makedirs(save_dir, exist_ok=True)
valset = LayeredDepth_Syn(mode = 'validation')
valloader = DataLoader(valset, batch_size=1, pin_memory=True, num_workers=4, drop_last=True)
progress_bar_val = tqdm(
            enumerate(valloader),
            total=len(valloader),
            ncols=100,
            dynamic_ncols=True
        )

for i, sample in progress_bar_val:
    
    img = sample['image'].cuda().float()
    depth_l1, valid_mask_l1 = sample['d1'].cuda(), sample['d1_valid_mask'].cuda()
    depth_l3, valid_mask_l3 = sample['d3'].cuda(), sample['d3_valid_mask'].cuda()
    depth_l5, valid_mask_l5 = sample['d5'].cuda(), sample['d5_valid_mask'].cuda()
    depth_l7, valid_mask_l7 = sample['d7'].cuda(), sample['d7_valid_mask'].cuda()
    with torch.no_grad():
        preds = model(img)

     # ✅ 只顯示第一筆
    # ✅ 只顯示第一筆樣本
    # ---- 1️⃣ 反標準化影像 ----
    img_vis = img[0].detach().cpu().permute(1, 2, 0).numpy()
    img_vis = np.clip((img_vis * np.array([0.229, 0.224, 0.225]) +
                    np.array([0.485, 0.456, 0.406])), 0, 1)
    img_vis = (img_vis[:, :, ::-1] * 255).astype(np.uint8)  # RGB→BGR for cv2

    # ---- 2️⃣ 預測與 GT 深度圖 ----
    pred_vis = [p[0].detach().cpu().numpy() for p in preds]
    gt_vis = [depth_l1[0].detach().cpu().numpy(),
            depth_l3[0].detach().cpu().numpy(),
            depth_l5[0].detach().cpu().numpy(),
            depth_l7[0].detach().cpu().numpy()]
    mask_vis = [valid_mask_l1[0].detach().cpu().numpy(),
                valid_mask_l3[0].detach().cpu().numpy(),
                valid_mask_l5[0].detach().cpu().numpy(),
                valid_mask_l7[0].detach().cpu().numpy()]

    # ---- 3️⃣ normalize depth & mask for visualization ----
    def colorize_depth(depth):
        d_norm = cv2.normalize(depth, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        return cv2.applyColorMap(d_norm, cv2.COLORMAP_PLASMA)

    def colorize_mask(mask):
        m = (mask * 255).astype(np.uint8)
        return cv2.cvtColor(m, cv2.COLOR_GRAY2BGR)

    pred_color = [colorize_depth(d) for d in pred_vis]
    gt_color = [colorize_depth(d) for d in gt_vis]
    mask_color = [colorize_mask(m) for m in mask_vis]

    # ---- 4️⃣ 拼接圖像 ----
    # ---- 4️⃣ 拼接圖像 ----
    blank = np.zeros_like(pred_color[0])  # 建立空白灰圖，大小與 depth 相同

    # 第一行：輸入影像 + 預測
    row1 = np.concatenate([img_vis] + pred_color, axis=1)

    # 第二行：空白 + GT 深度
    row2 = np.concatenate([blank] + gt_color, axis=1)

    # 第三行：空白 + Mask
    row3 = np.concatenate([blank] + mask_color, axis=1)
    vis_img = np.concatenate([row1, row2, row3], axis=0)

    # ---- 5️⃣ 儲存 ----
    save_path = os.path.join(save_dir, f"sample_{i:04d}.png")
    cv2.imwrite(save_path, vis_img)
    #print(f"✅ Saved visualization: {save_path}")


Resolving data files:   0%|          | 0/56 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/56 [00:00<?, ?it/s]

100%|██████████| 500/500 [02:57<00:00,  2.81it/s]


: 